In [156]:
# Import Libraries
import os 
import pandas as pd
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', 200)

# Import and Merge Historical Team Data 
mdcm = pd.read_csv('../data/mdcm/NCAA_Tourney_2002_2023.csv')
team_spellings = pd.read_csv('../data/mdcm/team_spellings.csv')

ncaa_sheets = pd.read_csv('../data/cbbdata/team/ncaa_sheets.csv').query('year != 2020')
selection_sunday_resume = pd.read_csv('../data/cbbdata/team/selection_sunday_resume.csv').query('year != 2020')

coach_results = pd.read_csv('../data/kaggle/march_madness_data/coach_results.csv')
barttovik_home = pd.read_csv('../data/kaggle/march_madness_data/barttovik_home.csv')
barttovik_away = pd.read_csv('../data/kaggle/march_madness_data/barttovik_away.csv')
kenpom_barttovik = pd.read_csv('../data/kaggle/march_madness_data/kenpom_barttovik.csv')
shooting_splits = pd.read_csv('../data/kaggle/march_madness_data/shooting_splits.csv')
heat_check = pd.read_csv('../data/kaggle/march_madness_data/heat_check.csv')

In [157]:
# Define Functions
def merge_team_season(df: pd.DataFrame, df_merge_onto: pd.DataFrame):
    df_both_teams = pd.DataFrame()
    for team_num in range(1, 3):
        
        # Adjust Column Names Due To Team1 and Team2
        df_merge_onto.columns = df_merge_onto.columns.str[6:] if team_num == 2 else df_merge_onto.columns
        df_merge_onto = df_merge_onto.add_prefix(f'team{team_num}_')
        
         # Original Merge
        df = pd.merge(df, df_merge_onto, how = 'left', left_on = [f'team{team_num}_teamname', 'season'], right_on = [f'team{team_num}_team', f'team{team_num}_year'])
        df_not_merged = df[df[f'team{team_num}_team'].isna() == True]
        
        # Remove Columns That Didn't Merge Properly
        neg_col_count_df_merge_onto = df_merge_onto.shape[1] * -1
        df_not_merged = df_not_merged.iloc[:, :neg_col_count_df_merge_onto]
        
        # Loop Through Columnns To Fix The Merge
        print(f"Merging Team {team_num} ...")
        merge_complete, i = False, 1
        while merge_complete == False:
            team_season_loop = pd.merge(df_not_merged, df_merge_onto, how = 'left', left_on = [f'team{team_num}_name_spelling_{i}', 'season'], right_on = [f'team{team_num}_team', f'team{team_num}_year'])
            print(f'Team Season Loop {i}:', len(team_season_loop), 'total rows.')
            
            matched_df = team_season_loop[team_season_loop[f'team{team_num}_team'].isna() == False]
            print('Matched DataFrame Rows:', len(matched_df)) 
            unmatched_df = team_season_loop[team_season_loop[f'team{team_num}_team'].isna() == True]
            print('Unmatched DataFrame Rows:', len(unmatched_df)) 
            
            # For The DataFrames With Data In The Column From Second DF, Add To team_season
            if len(matched_df) > 0:
                df = pd.concat([df, matched_df])
            # If There Are Still Null Rows, Throw Those Back In The Loop For The Next Iteration
            if len(unmatched_df) > 0:
                df_not_merged = unmatched_df.iloc[:, :neg_col_count_df_merge_onto]
                print('Not Merged Ending Iteration Shape:', df_not_merged.shape, '\n')
            # If There Aren't Any Null Rows Left, End The Loop
            else:
                print(f'\nTeam {team_num} Merge Complete!\n')
                merge_complete = True
            if i == 11:
                merge_complete = True 
            i += 1
        df_both_teams = pd.concat([df_both_teams, df])
            
    # Inspect That All Went As Planned
    for team in ['team1_id','team2_id']:
        for na_value in [True, False]:
            print('Team:', team, 'Null Values?', na_value)
            print('During & Post 2019: ', df_both_teams[(df_both_teams[team].isna() == na_value) & (df_both_teams['season'] >= 2019)].shape)
            print('Before 2019', df_both_teams[(df_both_teams[team].isna() == na_value) & (df_both_teams['season'] < 2019)].shape, '\n')
            
    return df_both_teams.drop(['team1_team','team1_year', 'team2_team', 'team2_year'], axis = 1)

In [158]:
# Adjust Team Spellings
team_spellings = team_spellings.pivot_table(index='team_id', columns=team_spellings.groupby('team_id').cumcount(), values='name_spelling', aggfunc='first')
team_spellings.columns = [f'name_spelling_{i + 1}' for i in range(team_spellings.shape[1])]
team_spellings.reset_index(inplace=True)

# Merge Team Spellings
team_spellings_t1 = team_spellings.add_prefix('team1_')
mdcm = pd.merge(mdcm, team_spellings_t1, how = 'inner', left_on = ['team1_id'], right_on = ['team1_team_id'])
team_spellings_t2 = team_spellings.add_prefix('team2_')
mdcm = pd.merge(mdcm, team_spellings_t2, how = 'inner', left_on = ['team2_id'], right_on = ['team2_team_id'])

# Merge MDCM and NCAA_Sheets (2019-2024)
team_season_2019 = mdcm[mdcm['season'] >= 2019]
ncaa_sheets.drop(['seed', 'net', 'quad_1a', 'quad_1', 'quad_2', 
                  'quad_1_and_2', 'quad_3', 'quad_4'], axis = 1, inplace = True)
team_season_fixed_merge = merge_team_season(team_season_2019, ncaa_sheets)
mdcm = pd.concat([team_season_fixed_merge, mdcm[mdcm['season'] < 2019]])

# Merge MDCM and Selection Sunday (2008-2024)
selection_sunday_resume = selection_sunday_resume[selection_sunday_resume['year'] != 'Year']
selection_sunday_resume['year'] = selection_sunday_resume['year'].astype(int)
selection_sunday_resume.drop(columns = ['net', 'seed'], axis = 1, inplace = True)
mdcm = merge_team_season(mdcm, selection_sunday_resume)

# Merge All Kaggle Datasets


Merging Team 1 ...
Team Season Loop 1: 54 total rows.
Matched DataFrame Rows: 0
Unmatched DataFrame Rows: 54
Not Merged Ending Iteration Shape: (54, 130) 

Team Season Loop 2: 54 total rows.
Matched DataFrame Rows: 0
Unmatched DataFrame Rows: 54
Not Merged Ending Iteration Shape: (54, 130) 

Team Season Loop 3: 54 total rows.
Matched DataFrame Rows: 5
Unmatched DataFrame Rows: 49
Not Merged Ending Iteration Shape: (49, 130) 

Team Season Loop 4: 49 total rows.
Matched DataFrame Rows: 7
Unmatched DataFrame Rows: 42
Not Merged Ending Iteration Shape: (42, 130) 

Team Season Loop 5: 42 total rows.
Matched DataFrame Rows: 10
Unmatched DataFrame Rows: 32
Not Merged Ending Iteration Shape: (32, 130) 

Team Season Loop 6: 32 total rows.
Matched DataFrame Rows: 29
Unmatched DataFrame Rows: 3
Not Merged Ending Iteration Shape: (3, 130) 

Team Season Loop 7: 3 total rows.
Matched DataFrame Rows: 1
Unmatched DataFrame Rows: 2
Not Merged Ending Iteration Shape: (2, 130) 

Team Season Loop 8: 2 tot

,team1_id,team1_score,team2_id,team2_score,WLoc,num_ot,team1_position,team2_position,team1_seed,team2_seed,strongseed,weakseed,team1_region,team2_region,slot,team1_teamname,team2_teamname,season,host,host_lat,host_long,team1_lat,team1_long,team2_lat,team2_long,team1_coach_id,team1_pt_school_ncaa,team1_pt_overall_ncaa,team1_pt_school_s16,team1_pt_overall_s16,team1_pt_school_ff,team1_pt_overall_ff,team1_pt_career_school_wins,team1_pt_career_school_losses,team1_pt_career_overall_wins,team1_pt_career_overall_losses,team1_pt_team_season_wins,team1_pt_team_season_losses,team1_pt_coach_season_wins,team1_pt_coach_season_losses,team2_coach_id,team2_pt_school_ncaa,team2_pt_overall_ncaa,team2_pt_school_s16,team2_pt_overall_s16,team2_pt_school_ff,team2_pt_overall_ff,team2_pt_career_school_wins,team2_pt_career_school_losses,team2_pt_career_overall_wins,team2_pt_career_overall_losses,team2_pt_team_season_wins,team2_pt_team_season_losses,team2_pt_coach_season_wins,team2_pt_coach_season_losses,team1_ap_final,team1_ap_preseason,team1_coaches_before_final,team1_coaches_preseason,team2_ap_final,team2_ap_preseason,team2_coaches_before_final,team2_coaches_preseason,team1_fg2pct,team1_fg3pct,team1_ftpct,team1_blockpct,team1_oppfg2pct,team1_oppfg3pct,team1_oppftpct,team1_oppblockpct,team1_f3grate,team1_oppf3grate,team1_arate,team1_opparate,team1_stlrate,team1_oppstlrate,team2_fg2pct,team2_fg3pct,team2_ftpct,team2_blockpct,team2_oppfg2pct,team2_oppfg3pct,team2_oppftpct,team2_oppblockpct,team2_f3grate,team2_oppf3grate,team2_arate,team2_opparate,team2_stlrate,team2_oppstlrate,team1_tempo,team1_adjtempo,team1_oe,team1_adjoe,team1_de,team1_adjde,team2_tempo,team2_adjtempo,team2_oe,team2_adjoe,team2_de,team2_adjde,game_id,team1_team_id,team1_name_spelling_1,team1_name_spelling_2,team1_name_spelling_3,team1_name_spelling_4,team1_name_spelling_5,team1_name_spelling_6,team1_name_spelling_7,team1_name_spelling_8,team1_name_spelling_9,team1_name_spelling_10,team1_name_spelling_11,team1_name_spelling_12,team2_team_id,team2_name_spelling_1,team2_name_spelling_2,team2_name_spelling_3,team2_name_spelling_4,team2_name_spelling_5,team2_name_spelling_6,team2_name_spelling_7,team2_name_spelling_8,team2_name_spelling_9,team2_name_spelling_10,team2_name_spelling_11,team2_name_spelling_12,team1_rank,team1_kpi_resume,team1_sor_resume,team1_avg_resume,team1_bpi_quality,team1_kp_quality,team1_sag_quality,team1_avg_quality,team2_rank,team2_kpi_resume,team2_sor_resume,team2_avg_resume,team2_bpi_quality,team2_kp_quality,team2_sag_quality,team2_avg_quality
0,1314,81,1181,77,N,0,W08,X02,8,2,2,8,W,X,R5WX,North Carolina,Duke,2022,new_orleans,29.9667,-90.0500,35.912165,-79.050969,36.001590,-78.942260,hubert-davis-1,0.0,0.0,0.0,0.0,0.0,0.0,24.0,9,24.0,9,24.0,9,24.0,9,mike-krzyzewski-1,35.0,35.0,25.0,25.0,12.0,12.0,1125.0,308,1198.0,367,28.0,6,28.0,6,NaN,19.0,NaN,20.0,9.0,9.0,10.0,9.0,50.714286,36.206897,77.219430,9.814529,48.299845,34.920635,73.696145,8.492063,37.437934,36.878049,53.618421,48.031496,0.076692,0.093829,55.844156,36.821192,72.881356,13.258110,46.897038,31.396957,67.160494,8.097785,36.579457,33.769267,56.987116,51.793722,0.094448,0.091014,70.2181,70.1745,109.416,113.035,100.9740,96.9911,68.2765,67.4185,117.1520,119.3570,98.2346,95.6444,2022-1314-1181,1314,unc,north-carolina,north carolina,n carolina,N Carolina,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1181,duke,Duke,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,31,34,19,26.5,26,29,25,26.7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1277,68,1181,67,N,0,W02,W01,2,1,1,2,W,W,R4W1,Michigan St,Duke,2019,washington,38.8951,-77.0367,42.724760,-84.473639,36.001590,-78.942260,tom-izzo-1,21.0,21.0,13.0,13.0,7.0,7.0,602.0,231,602.0,231,28.0,6,28.0,6,mike-krzyzewski-1,34.0,34.0,24.0,24.0,12.0,12.0,1056.0,284,1129.0,343,29.0,5,29.0,5,11.0,10.0,5.0,10.0,4.0,3.0,1.0,4.0,55.140187,38.342697,75.000000,13.899614,41.158301,31.781915,67.543860,8.071368,37.691900,36.736688,68.329718,52.331606,0.077123,0.093336,58.302583,30.195599,68.974700,16.361072,44.922426,29.349270,6

In [ ]:
# Merge Kaggle march_madness_data
for file in ['coach_results', 'barttovik_home', 'barttovik_away', 'kenpom_barttovik', 'shooting_splits', 'heat_check']:
    if file == 'coach_results':
        # need additional param for coach id
        # coach_results = pd.merge(mdcm, pd.read_csv(f'../data/kaggle/march_madness_data/{file}.csv'))
    elif file in 

In [168]:
file = ['coach_results', 'barttovik_home', 'barttovik_away', 'kenpom_barttovik', 'shooting_splits', 'heat_check']

pd.read_csv(f'../data/kaggle/march_madness_data/{file[3]}.csv').head()


,YEAR,CONF,CONF ID,QUAD NO,QUAD ID,TEAM NO,TEAM ID,TEAM,SEED,ROUND,K TEMPO,K TEMPO RANK,KADJ T,KADJ T RANK,K OFF,KO RANK,KADJ O,KADJ O RANK,K DEF,KD RANK,KADJ D,KADJ D RANK,KADJ EM,KADJ EM RANK,BADJ EM,BADJ O,BADJ D,BARTHAG,GAMES,W,L,WIN%,EFG%,EFG%D,FTR,FTRD,TOV%,TOV%D,OREB%,DREB%,OP OREB%,OP DREB%,RAW T,2PT%,2PT%D,3PT%,3PT%D,BLK%,BLKED%,AST%,OP AST%,2PTR,3PTR,2PTRD,3PTRD,BADJ T,AVG HGT,EFF HGT,EXP,TALENT,FT%,OP FT%,PPPO,PPPD,ELITE SOS,WAB,BADJ EM RANK,BADJ O RANK,BADJ D RANK,BARTHAG RANK,EFG% RANK,EFGD% RANK,FTR RANK,FTRD RANK,TOV% RANK,TOV%D RANK,OREB% RANK,DREB% RANK,OP OREB% RANK,OP DREB% RANK,RAW T RANK,2PT% RANK,2PT%D RANK,3PT% RANK,3PT%D RANK,BLK% RANK,BLKED% RANK,AST% RANK,OP AST% RANK,2PTR RANK,3PTR RANK,2PTRD RANK,3PTRD RANK,BADJT RANK,AVG HGT RANK,EFF HGT RANK,EXP RANK,TALENT RANK,FT% RANK,OP FT% RANK,PPPO RANK,PPPD RANK,ELITE SOS RANK
0,2024,MAC,17,64,4,1079,2,Akron,14,0,66.5246,292,65.4089,294,109.684,88,109.690,110,97.9075,40,100.6460,72,9.04390,90,8.267,108.551,100.284,0.713,21,16,5,76.19048,53.0,48.4,34.6,29.8,16.9,17.9,29.7,74.7,25.3,70.3,66.5,54.4,49.7,34.0,30.8,8.0,8.3,49.1,47.1,58.5,41.5,63.8,36.2,65.326,76.612,79.890,2.564,11.033,73.0,68.6,1.098,0.980,12.238,0.0,95,129,71,95,73,79,132,104,154,130,157,40,40,157,295,43,148,170,37,233,108,212,93,288,77,227,139,298,266,216,19,175,125,46,87,42,277
1,2024,SEC,28,62,2,1078,3,Alabama,3,0,73.0676,24,72.0976,17,123.229,1,125.857,2,102.6930,137,99.7332,57,26.12390,7,27.305,126.633,99.328,0.942,22,16,6,72.72727,57.7,48.6,36.3,35.8,16.9,16.9,35.0,71.0,29.0,65.0,73.2,58.1,49.2,38.2,31.7,10.7,11.3,51.4,44.5,52.1,47.9,62.9,37.1,72.011,78.251,82.491,2.070,26.153,79.1,70.6,1.228,1.028,31.268,0.0,5,2,53,6,3,88,93,261,154,205,29,174,174,29,23,9,125,16,70,108,314,140,41,352,13,197,176,18,29,7,162,110,5,138,1,140,7
2,2024,SB,26,63,3,1077,7,Appalachian St.,12,0,68.9060,171,67.1149,216,108.813,109,108.259,134,96.0406,24,98.5874,45,9.67158,87,7.980,106.628,98.648,0.710,21,17,4,80.95238,52.0,45.4,32.1,21.0,15.1,15.4,30.1,72.4,27.6,69.9,68.9,54.3,44.1,31.5,31.9,15.6,5.7,50.5,43.3,66.7,33.3,66.3,33.7,66.932,76.968,80.693,2.129,16.566,66.2,69.0,1.088,0.965,12.355,0.0,98,163,47,96,115,16,199,5,61,300,142,115,115,142,176,44,13,279,79,8,3,165,26,88,278,300,65,221,209,102,149,147,328,63,106,29,274
3,2024,P12,24,61,1,1076,8,Arizona,2,0,74.5598,8,72.7631,11,118.039,10,121.455,6,95.9483,23,94.4483,12,27.00700,4,26.191,121.091,94.900,0.943,22,17,5,77.27273,54.8,49.3,36.0,24.6,16.3,19.1,37.8,78.1,21.9,62.2,74.6,54.8,47.5,36.5,34.7,9.1,7.7,59.1,54.4,68.1,31.9,60.3,39.7,72.836,78.144,81.690,1.916,70.181,72.3,71.1,1.181,0.956,29.109,0.0,6,8,14,5,31,121,104,16,120,72,9,3,3,9,8,33,77,50,247,186,69,31,276,58,307,99,267,11,40,22,210,11,163,167,10,23,22
4,2024,SEC,28,64,4,1075,12,Auburn,5,0,70.7523,82,69.8715,71,116.363,13,119.147,13,93.0710,5,92.3563,5,26.79030,5,27.394,120.062,92.668,0.952,22,18,4,81.81818,53.3,43.3,36.2,36.5,15.0,19.4,34.0,71.0,29.0,66.0,70.8,55.1,42.6,33.6,29.7,15.7,7.8,63.5,45.9,61.0,39.0,64.4,35.6,69.980,77.653,80.941,2.196,44.191,76.3,73.2,1.164,0.931,22.047,0.0,4,12,6,4,62,2,96,276,56,64,42,174,174,42,82,28,3,186,18,7,77,6,63,228,138,244,121,63,101,78,127,64,37,275,14,6,86


In [167]:
file[1:]

['barttovik_home', 'barttovik_away']

In [ ]:
# Import Team Ratings By Day (2015-2019)
team_ratings = pd.read_csv('../data/cbbdata/team/team_ratings.csv').query('year != 2020')
team_ratings['date']= pd.to_datetime(team_ratings['date'])

# Split Daily Team Ranknings By Before/After Selection Sunday
ss_dict = {2024: "2024-3-17", 2023: "2023-3-12", 2022: "2022-3-13", 
           2021: "2021-3-14", 2019: "2019-3-17", 2018: "2018-3-11", 
           2017: "2017-3-12", 2016: "2016-3-13", 2015: "2015-3-15"}

# Create A Pre and During NCAA Tournament Day By Day Ratings
team_rating_pre_ncaa = pd.DataFrame(columns = team_ratings.columns)
team_rating_ncaa = pd.DataFrame(columns = team_ratings.columns)
for year, ss_date in ss_dict.items():
    team_rating_pre_ncaa = pd.concat([team_rating_pre_ncaa, team_ratings[(team_ratings['year'] == year) & (team_ratings['date'] < ss_date)]])    
    team_rating_ncaa = pd.concat([team_rating_ncaa, team_ratings[(team_ratings['year'] == year) & (team_ratings['date'] > ss_date)]])
    
# Look At Team Rating By Day and Calculate Rolling Adj Offensive Rank
team_rating_pre_ncaa['rolling_avg_adj_o_rk'] = team_rating_pre_ncaa.groupby(['team', 'year'])['adj_o'].transform(lambda x: x.rolling(window=3).mean())
df = team_rating_pre_ncaa[(team_rating_pre_ncaa['team'] == 'Kentucky') & (team_rating_pre_ncaa['year'] == 2015)]
# team_rating_ncaa['rolling_avg_adj_o_rk'] = team_rating_ncaa.groupby(['team', 'year'])['adj_o'].transform(lambda x: x.rolling(window=3).mean())
# df = team_rating_ncaa[(team_rating_ncaa['team'] == 'Kentucky') & (team_rating_ncaa['year'] == 2015)]